[參考](https://huggingface.co/learn/nlp-course/zh-TW/chapter3/1?fw=pt)

**處理數據**

這一小節學習[第一小節](https://huggingface.co/course/chapter2)中提到的「如何使用模型中心（hub）大型數據集」，下面是我們用模型中心的數據在 PyTorch 上訓練句子分類器的一個例子：

In [ ]:
!pip install accelerate -U

要在 Runtime --> restart session

執行以下會遇到問題。請參考[這裏](https://discuss.huggingface.co/t/trainingargument-does-not-work-on-colab/43372/5)

In [ ]:
import torch
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification

# Same as before
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
sequences = [
    "I've been waiting for a HuggingFace course my whole life.",
    "This course is amazing!",
]
batch = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")

# This is new
batch["labels"] = torch.tensor([1, 1])

optimizer = AdamW(model.parameters())
loss = model(**batch).loss
loss.backward()
optimizer.step()

當然，僅僅用兩句話訓練模型不會產生很好的效果。為了獲得更好的結果，您需要準備一個更大的數據集。

在本節中，我們將使用MRPC（微軟研究釋義語料庫）數據集作為示例，該數據集由威廉·多蘭和克里斯·布羅克特在這篇文章發佈。該數據集由5801對句子組成，每個句子對帶有一個標籤，指示它們是否為同義（即，如果兩個句子的意思相同）。我們在本章中選擇了它，因為它是一個小數據集，所以很容易對它進行訓練。

**從模型中心（Hub）加載數據集**

模型中心（hub）不只是包含模型；它也有許多不同語言的多個[數據集](https://huggingface.co/datasets)。點擊數據集的鏈接即可進行瀏覽。我們建議您在閱讀本節後閱讀一下[加載和處理新的數據集](https://huggingface.co/docs/datasets/loading_datasets.html#from-the-huggingface-hub)這篇文章，這會讓您對huggingface的darasets更加清晰。但現在，讓我們使用MRPC數據集中的[GLUE 基準測試數據集](https://gluebenchmark.com/)，它是構成MRPC數據集的10個數據集之一，這是一個學術基準，用於衡量機器學習模型在10個不同文本分類任務中的性能。

🤗 Datasets庫提供了一個非常便捷的命令，可以在模型中心（hub）上下載和緩存數據集。我們可以通過以下的代碼下載MRPC數據集：

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

raw_datasets = load_dataset("glue", "mrpc")
raw_datasets

In [ ]:
raw_train_dataset = raw_datasets["train"]
raw_train_dataset[0]

我們可以看到標籤已經是整數了，所以我們不需要對標籤做任何預處理。要知道哪個數字對應於哪個標籤，我們可以查看raw_train_dataset的features. 這將告訴我們每列的類型：

In [ ]:
raw_train_dataset.features

為了預處理數據集，我們需要將文本轉換為模型能夠理解的數字。正如你在[第二章](https://huggingface.co/course/chapter2)上看到的那樣

In [ ]:
from transformers import AutoTokenizer

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenized_sentences_1 = tokenizer(raw_datasets["train"]["sentence1"])
tokenized_sentences_2 = tokenizer(raw_datasets["train"]["sentence2"])

然而，在兩句話傳遞給模型，預測這兩句話是否是同義之前。我們需要這兩句話依次進行適當的預處理。幸運的是，標記器不僅僅可以輸入單個句子還可以輸入一組句子，並按照我們的BERT模型所期望的輸入進行處理：

In [ ]:
inputs = tokenizer("This is the first sentence.", "This is the second one.")
inputs

我們在[第二章](https://huggingface.co/course/chapter2) 討論了輸入詞id(input_ids) 和 注意力遮罩(attention_mask) ，但我們在那個時候沒有討論類型標記ID(token_type_ids)。在這個例子中，類型標記ID(token_type_ids)

如果我們將input_ids中的id轉換回文字:

In [ ]:
tokenizer.convert_ids_to_tokens(inputs["input_ids"])

所以我們看到模型需要輸入的形式是 [CLS] sentence1 [SEP] sentence2 [SEP]。因此，當有兩句話的時候。類型標記ID(token_type_ids) 的值是：

In [ ]:
inputs['token_type_ids']


如您所見，輸入中 [CLS] sentence1 [SEP] 它們的類型標記ID均為0，而其他部分，對應於sentence2 [SEP]，所有的類型標記ID均為1.

請注意，如果選擇其他的檢查點，則不一定具有類型標記ID(token_type_ids)（例如，如果使用DistilBERT模型，就不會返回它們）。只有當它在預訓練期間使用過這一層，模型在構建時依賴它們，才會返回它們。

用類型標記ID對BERT進行預訓練,並且使用[第一章](https://huggingface.co/course/chapter1)的遮罩語言模型，還有一個額外的應用類型，叫做下一句預測. 這項任務的目標是建立成對句子之間關係的模型。

在下一個句子預測任務中，會給模型輸入成對的句子（帶有隨機遮罩的標記），並被要求預測第二個句子是否緊跟第一個句子。為了提高模型的泛化能力，數據集中一半的兩個句子在原始文檔中挨在一起，另一半的兩個句子來自兩個不同的文檔。

一般來說，你不需要擔心是否有類型標記ID(token_type_ids)。在您的標輸入中：只要您對標記器和模型使用相同的檢查點，一切都會很好，因為標記器知道向其模型提供什麼。

現在我們已經瞭解了標記器如何處理一對句子，我們可以使用它對整個數據集進行處理：如[之前的章節](https://huggingface.co/course/chapter2)，我們可以給標記器提供一組句子，第一個參數是它第一個句子的列表，第二個參數是第二個句子的列表。這也與我們在[第二](https://huggingface.co/course/chapter2)章中看到的填充和截斷選項兼容. 因此，預處理訓練數據集的一種方法是：

In [ ]:
tokenized_dataset = tokenizer(
    raw_datasets["train"]["sentence1"],
    raw_datasets["train"]["sentence2"],
    padding=True,
    truncation=True,
)

這很有效，但它的缺點是返回字典（字典的鍵是輸入詞id(input_ids) ， 注意力遮罩(attention_mask) 和 類型標記ID(token_type_ids)，字典的值是鍵所對應值的列表）。而且只有當您在轉換過程中有足夠的內存來存儲整個數據集時才不會出錯（而🤗數據集庫中的數據集是以[Apache Arrow](https://arrow.apache.org/)文件存儲在磁盤上，因此您只需將接下來要用的數據加載在內存中，因此會對內存容量的需求要低一些）。

為了將數據保存為數據集，我們將使用 [Dataset.map()](https://huggingface.co/docs/datasets/v2.15.0/en/package_reference/main_classes#datasets.Dataset.mapp)方法，如果我們需要做更多的預處理而不僅僅是標記化，那麼這也給了我們一些額外的自定義的方法。這個方法的工作原理是在數據集的每個元素上應用一個函數，因此讓我們定義一個標記輸入的函數：

In [ ]:
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

此函數的輸入是一個字典（與數據集的項類似），並返回一個包含輸入詞id(input_ids) ， 注意力遮罩(attention_mask) 和 類型標記ID(token_type_ids) 鍵的新字典。請注意，如果像上面的示例一樣，如果鍵所對應的值包含多個句子（每個鍵作為一個句子列表），那麼它依然可以工作，就像前面的例子一樣標記器可以處理成對的句子列表。這樣的話我們可以在調用map()使用該選項 batched=True ，這將顯著加快標記與標記的速度。這個標記器來自🤗 [Tokenizers](https://github.com/huggingface/tokenizers)庫由Rust編寫而成。當我們一次給它大量的輸入時，這個標記器可以非常快。

請注意，我們現在在標記函數中省略了padding參數。這是因為在標記的時候將所有樣本填充到最大長度的效率不高。一個更好的做法：在構建批處理時填充樣本更好，因為這樣我們只需要填充到該批處理中的最大長度，而不是整個數據集的最大長度。當輸入長度變化很大時，這可以節省大量時間和處理能力!

下面是我們如何在所有數據集上同時應用標記函數。我們在調用map時使用了batch =True，這樣函數就可以同時應用到數據集的多個元素上，而不是分別應用到每個元素上。這將使我們的預處理快許多

In [ ]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets

在使用預處理函數map()時，甚至可以通過傳遞num_proc參數使用並行處理。我們在這裡沒有這樣做，因為🤗標記器庫已經使用多個線程來更快地標記我們的樣本，但是如果您沒有使用該庫支持的快速標記器，使用num_proc可能會加快預處理。

我們的標記函數(tokenize_function)返回包含輸入詞id(input_ids) ， 注意力遮罩(attention_mask) 和 類型標記ID(token_type_ids) 鍵的字典,所以這三個字段被添加到數據集的標記的結果中。注意，如果預處理函數map()為現有鍵返回一個新值，那將會修改原有鍵的值。

最後一件我們需要做的事情是，當我們一起批處理元素時，將所有示例填充到最長元素的長度——我們稱之為動態填充。

負責在批處理中將數據整理為一個batch的函數稱為collate函數。它是你可以在構建DataLoader時傳遞的一個參數，默認是一個函數，它將把你的數據集轉換為PyTorch張量，並將它們拼接起來(如果你的元素是列表、元組或字典，則會使用遞歸)。這在我們的這個例子中下是不可行的，因為我們的輸入不是都是相同大小的。我們故意在之後每個batch上進行填充，避免有太多填充的過長的輸入。這將大大加快訓練速度，但請注意，如果你在TPU上訓練，這可能會導致問題——TPU喜歡固定的形狀，即使這需要額外的填充。

為了解決句子長度統一的問題，我們必須定義一個collate函數，該函數會將每個batch句子填充到正確的長度。幸運的是，🤗transformer庫通過DataCollatorWithPadding為我們提供了這樣一個函數。當你實例化它時，需要一個標記器(用來知道使用哪個詞來填充，以及模型期望填充在左邊還是右邊)，並將做你需要的一切:

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

為了測試這個新玩具，讓我們從我們的訓練集中抽取幾個樣本。這裡，我們刪除列idx, sentence1和sentence2，因為不需要它們，並查看一個batch中每個條目的長度:

In [ ]:
samples = tokenized_datasets["train"][:8]
samples = {k: v for k, v in samples.items() if k not in ["idx", "sentence1", "sentence2"]}
[len(x) for x in samples["input_ids"]]

毫無疑問，我們得到了不同長度的樣本，從32到67。動態填充意味著該批中的所有樣本都應該填充到長度為67，這是該批中的最大長度。如果沒有動態填充，所有的樣本都必須填充到整個數據集中的最大長度，或者模型可以接受的最大長度。讓我們再次檢查data_collator是否正確地動態填充了這批樣本：

In [ ]:
batch = data_collator(samples)
{k: v.shape for k, v in batch.items()}

看起來不錯！現在，我們已經將原始文本轉化為了模型可以處理的數據，我們已準備好對其進行微調！

==================休息一下=================

Transformers提供了一個 Trainer 類來幫助您在自己的數據集上微調任何預訓練模型。完成上一節中的所有數據預處理工作後，您只需要執行幾個步驟來創建 Trainer .最難的部分可能是為 Trainer.train()配置運行環境，因為它在 CPU 上運行速度會非常慢。如果您沒有設置 GPU，您可以訪問免費的 GPU 或 TPUGoogle Colab.

下面的示例假設您已經執行了上一節中的示例。下面這段代碼，概括了您需要提前運行的代碼：

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
data_collator.tokenizer.name_or_path

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")

第二步是定義我們的模型。正如在之前的章節一樣，我們將使用 AutoModelForSequenceClassification 類，它有兩個參數：

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

你會注意到，和[第二章](https://huggingface.co/course/chapter2)不一樣的是，在實例化此預訓練模型後會收到警告。這是因為 BERT 沒有在句子對分類方面進行過預訓練，所以預訓練模型的頭部已經被丟棄，而是添加了一個適合句子序列分類的新頭部。警告表明一些權重沒有使用（對應於丟棄的預訓練頭的那些），而其他一些權重被隨機初始化（新頭的那些）。最後鼓勵您訓練模型，這正是我們現在要做的。

一旦我們有了我們的模型，我們就可以定義一個 Trainer 通過將之前構造的所有對象傳遞給它——我們的model 、training_args ，訓練和驗證數據集，data_collator ，和 tokenizer ：

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

請注意，當您在這裡完成tokenizer後，默認 Trainer使用 的data_collator會使用之前預定義的 DataCollatorWithPadding ，因此您可以在這個例子中跳過 data_collator=data_collator。在第 2 節中向您展示這部分處理仍然很重要！

為了讓預訓練模型在在我們的數據集上微調，我們只需要調用Trainer的train() 方法 ：

In [ ]:
trainer.train()

這將開始微調（在GPU上應該需要幾分鐘），並每500步報告一次訓練損失。但是，它不會告訴您模型的性能如何（或質量如何）。這是因為:

1.   我們沒有通過將evaluation_strategy設置為“steps”(在每次更新參數的時候評估)或“epoch”(在每個epoch結束時評估)來告訴Trainer在訓練期間進行評估。
2.   我們沒有為Trainer提供一個compute_metrics()函數來直接計算模型的好壞(否則評估將只輸出loss，這不是一個非常直觀的數字)。

**評估**

讓我們看看如何構建一個有用的 compute_metrics() 函數並在我們下次訓練時使用它。該函數必須採用 EvalPrediction 對象（帶有 predictions 和 label_ids 字段的參數元組）並將返回一個字符串到浮點數的字典（字符串是返回的指標的名稱，而浮點數是它們的值）。我們可以使用 Trainer.predict() 命令來使用我們的模型進行預測：

In [ ]:
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

predict() 的輸出結果是具有三個字段的命名元組： predictions , label_ids ， 和 metrics .這 metrics 字段將只包含傳遞的數據集的loss，以及一些運行時間（預測所需的總時間和平均時間）。如果我們定義了自己的 compute_metrics() 函數並將其傳遞給 Trainer ，該字段還將包含compute_metrics()的結果。

如你看到的， predictions 是一個形狀為 408 x 2 的二維數組（408 是我們使用的數據集中元素的數量）。這些是我們傳遞給predict()的數據集的每個元素的結果(logits)（正如你在之前的章節看到的情況）。要將我們的預測的可以與真正的標籤進行比較，我們需要在第二個軸上取最大值的索引：

In [ ]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

In [ ]:
### 需要安裝
!pip install evaluate

現在建立我們的 compute_metric() 函數來較為直觀地評估模型的好壞，我們將使用 🤗 [Evaluate](https://github.com/huggingface/evaluate/) 庫中的指標。我們可以像加載數據集一樣輕鬆加載與 MRPC 數據集關聯的指標，這次使用 evaluate.load() 函數。返回的對象有一個 compute()方法我們可以用來進行度量計算的方法：

In [ ]:
import evaluate

metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

您獲得的確切結果可能會有所不同，因為模型頭的隨機初始化可能會影響最終建立的模型。在這裡，我們可以看到我們的模型在驗證集上的準確率為 85.78%，F1 分數為 89.97。這是用於評估 GLUE 基準的 MRPC 數據集結果的兩個指標。而在[BERT 論文](https://arxiv.org/pdf/1810.04805.pdf)中展示的基礎模型的 F1 分數為 88.9。那是 uncased 模型，而我們目前正在使用 cased 模型，通過改進得到了更好的結果。

最後將所有東西打包在一起，我們得到了我們的 compute_metrics() 函數：

In [ ]:
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

為了查看模型在每個訓練週期結束的好壞，下面是我們如何使用compute_metrics()函數定義一個新的 Trainer ：

In [ ]:
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

請注意，我們設置了了一個新的 TrainingArguments 它的evaluation_strategy 設置為 epoch 並創建了一個新模型。如果不創建新的模型就直接訓練，就只會繼續訓練之前我們已經訓練過的模型。要啟動新的訓練運行，我們執行：

In [ ]:
trainer.train()

這一次，它將在訓練loss之外，還會輸出每個 epoch 結束時的驗證loss和指標。同樣，由於模型的隨機頭部初始化，您達到的準確率/F1 分數可能與我們發現的略有不同，但它應該在同一範圍內。

這 Trainer 將在多個 GPU 或 TPU 上開箱即用，並提供許多選項，例如混合精度訓練（在訓練的參數中使用 fp16 = True ）。我們將在第 10 章討論它支持的所有內容。

使用Trainer API微調的介紹到此結束。對最常見的 NLP 任務執行此操作的示例將在第 7 章中給出，但現在讓我們看看如何在純 PyTorch 中執行相同的操作。

==================休息一下====================


**一個完整的訓練**

現在，我們將瞭解如何在不使用Trainer類的情況下獲得與上一節相同的結果。同樣，我們假設您已經學習了第 2 節中的數據處理。下面是一個簡短的總結，涵蓋了您需要的所有內容:

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

訓練前的準備
在實際編寫我們的訓練循環之前，我們需要定義一些對象。第一個是我們將用於迭代批次的數據加載器。我們需要對我們的tokenized_datasets做一些處理，來處理Trainer自動為我們做的一些事情。具體來說，我們需要:

*   刪除與模型不期望的值相對應的列（如sentence1和sentence2列）。
*   將列名label重命名為labels（因為模型期望參數是labels）。
*   設置數據集的格式，使其返回 PyTorch 張量而不是列表。
針對上面的每個步驟，我們的 tokenized_datasets 都有一個方法:

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

然後，我們可以檢查結果中是否只有模型能夠接受的列:

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
)

為了快速檢驗數據處理中沒有錯誤，我們可以這樣檢驗其中的一個批次:

In [ ]:
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

請注意，實際的形狀可能與您略有不同，因為我們為訓練數據加載器設置了shuffle=True，並且模型會將句子填充到batch中的最大長度。

現在我們已經完全完成了數據預處理（對於任何 ML 從業者來說都是一個令人滿意但難以實現的目標），讓我們將注意力轉向模型。我們完全像在上一節中所做的那樣實例化它:

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

為了確保訓練過程中一切順利，我們將batch傳遞給這個模型:

In [ ]:
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

當我們提供 labels 時， 🤗 Transformers 模型都將返回這個batch的loss，我們還得到了 logits(batch中的每個輸入有兩個，所以張量大小為 8 x 2)。

我們幾乎準備好編寫我們的訓練循環了！我們只是缺少兩件事：優化器和學習率調度器。由於我們試圖自行實現 Trainer的功能，我們將使用相同的優化器和學習率調度器。Trainer 使用的優化器是 AdamW , 與 Adam 相同，但在權重衰減正則化方面有所不同(參見“[Decoupled Weight Decay Regularization](https://arxiv.org/abs/1711.05101)”作者:Ilya Loshchilov 和 Frank Hutter):

In [ ]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

最後，默認使用的學習率調度器只是從最大值 (5e-5) 到 0 的線性衰減。 為了定義它，我們需要知道我們訓練的次數，即所有數據訓練的次數(epochs)乘以的數據量（這是我們所有訓練數據的數量）。Trainer默認情況下使用三個epochs，因此我們定義訓練過程如下:

In [ ]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

**訓練循環**

最後一件事：如果我們可以訪問 GPU,我們將希望使用 GPU(在 CPU 上，訓練可能需要幾個小時而不是幾分鐘)。為此，我們定義了一個 device,它在GPU可用的情況下指向GPU 我們將把我們的模型和batche放在device上:

In [ ]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

我們現在準備好訓練了！為了瞭解訓練何時結束，我們使用 tqdm 庫,在訓練步驟數上添加了一個進度條:

In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

您可以看到訓練循環的核心與介紹中的非常相似。我們沒有要求任何檢驗，所以這個訓練循環不會告訴我們任何關於模型目前的狀態。我們需要為此添加一個評估循環。

**評估循環**

正如我們之前所做的那樣，我們將使用 🤗 Evaluate 庫提供的指標。我們已經瞭解了 metric.compute() 方法，當我們使用 add_batch()方法進行預測循環時，實際上該指標可以為我們累積所有 batch 的結果。一旦我們累積了所有 batch ，我們就可以使用 metric.compute() 得到最終結果 .以下是在評估循環中實現所有這些的方法:

In [ ]:
!pip install evaluate

In [ ]:
import evaluate

metric = evaluate.load("glue", "mrpc")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

同樣，由於模型頭部初始化和數據改組的隨機性，您的結果會略有不同，但它們應該在同一個範圍內。

**微調，檢查!**

這是非常令人高興的！ 在前兩章中，您瞭解了模型和標記器(tokenizer)，現在您知道如何針對您自己的數據對它們進行微調。回顧一下，在本章中，您：

*   瞭解了Hub中的數據集
*   學習瞭如何加載和預處理數據集，包括使用動態填充和整理器
*   實現您自己的模型微調和評估
*   實施了一個較為底層的訓練循環
*   使用 🤗 Accelerate 輕鬆調整您的訓練循環，使其適用於多個 GPU 或 TPU